In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    get_scheduler
)
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report
import torch


In [ ]:
train_data=pd.read_csv('/kaggle/input/data-set1/indic-health-demo-main/Dataset/IHQID-WebMD/train.csv')[['question_english', 'Manual_Intent']]
test_data=pd.read_csv('/kaggle/input/data-set1/indic-health-demo-main/Dataset/IHQID-WebMD/test.csv')[['question_english', 'Manual_Intent']]
label_encoder=LabelEncoder()
train_data['Encoded']=label_encoder.fit_transform(train_data['Manual_Intent'])
train_data.drop(columns=['Manual_Intent'],inplace=True)
test_data['Encoded']=label_encoder.transform(test_data['Manual_Intent'])
test_data.drop(columns=['Manual_Intent'],inplace=True)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model_checkpoint = "roberta-base"

hyper_parameters = {
    'batch_size': 8,
    'lr': 3e-5,
    'epochs': 10
}

In [ ]:
  tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

encoded_input_train = tokenizer.batch_encode_plus(  train_data['question_english'],
    add_special_tokens=True,
    padding=True,
    return_tensors='pt'
)

encoded_input_test = tokenizer.batch_encode_plus(
    test_data['question_english'],
    add_special_tokens=True,
    padding=True,
    return_tensors='pt'
)

train_dataloader = DataLoader(
    TensorDataset(
        encoded_input_train['input_ids'].to(device),
        encoded_input_train['attention_mask'].to(device),
        torch.tensor(train_data['Manual_Intent_Encoded']).to(device)
    ),
    batch_size=hyper_parameters['batch_size']
)

test_dataloader = DataLoader(
    TensorDataset(
        encoded_input_test['input_ids'].to(device),
        encoded_input_test['attention_mask'].to(device),
        torch.tensor(test_data['Manual_Intent_Encoded']).to(device)
    ),
    batch_size=hyper_parameters['batch_size']
)


In [ ]:
model = RobertaForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_encoder.classes_)
)
model.to(device)

optimizer = AdamW(
    model.parameters(),
    lr=hyper_parameters['lr']
)

lr_scheduler = get_scheduler(
  "linear",
  optimizer=optimizer,
     num_warmup_steps=0,
  num_training_steps=hyper_parameters['epochs'] * len(train_dataloader)
)

In [ ]:
model.train()

updater = tqdm(range(hyper_parameters['epochs']))
for epoch in updater:
    total_train_loss = 0.0
    for batch in train_dataloader:
        
        optimizer.zero_grad()
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2],
        }
        
        outputs = model(**inputs)
        
        loss = outputs.loss
        loss.backward()
         optimizer.step()
        lr_scheduler.step()
        
        total_train_loss += loss.item() 
    print("Epoch:", epoch + 1, " - Training Loss:", round(total_train_loss / len(train_dataloader), 4))
torch.save(model.state_dict(), f'ir_rob_en.model')
